In [1]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold



In [2]:
data = pd.read_csv('MFTrend_data.csv')
data

Company        Time  Total Assets  Total Liabilities  Total Equity  \
0       CINF  2019-12-01        3473.0             1442.0        2031.0   
1       SIRI  2015-12-01        -322.4             1154.0       -1476.3   
2        SNA  2016-12-01         392.1              187.6         204.5   
3       ATNI  2018-12-01         -98.3              -91.4          -6.9   
4        DRI  2019-05-01         423.2              225.4         197.8   
5        ABC  2017-09-01        1679.0             1743.9         -64.9   
6        HPQ  2013-10-01       -3092.0            -7915.0        4823.0   
7        SJM  2012-04-01         790.6              919.6        -129.0   
8        WHR  2018-12-01       -1691.0              232.0       -1923.0   
9        SNA  2017-12-01         525.9              188.8         337.1   
10       WAT  2012-12-01         445.0              204.1         240.8   
11        FE  2018-12-01       -2194.0            -5083.0        2889.0   
12       HFC  2019-12-01        1170.2             1119.9          50.3   
13       BTI  2019-12-01         230.4            -1022.3        1252.7   
14        MA  2015-12-01         921.0             1683.0        -762.0   
15       CVS  2018-12-01      101325.0            80477.0       20848.0   
16       NOC  2013-12-01        -162.0            -1268.0        1106.0   
17       ALL  2014-12-01      -15041.0           -15865.0         824.0   
18       TAP  2017-12-01         905.4             -660.2        1565.6   
19      AVGO  2014-11-01        7076.0             6719.0         357.0   
20      QCOM  2019-09-01         239.0            -3863.0        4102.0   
21       UTL  2011-12-01          87.1               84.4           2.7   
22      REGN  2017-12-01        1790.8               96.0        1694.9   
23       BLK  2011-12-01        1437.0             2409.0        -972.0   
24      XLNX  2017-04-01         -78.8                3.5         -82.3   
25      ULTA  2019-02-01         282.5              236.5          46.0   
26       DAR  2017-12-01         260.2                8.7         251.5   
27       PWR  2013-12-01         652.4              183.1         469.4   
28       KSU  2014-12-01         541.0              153.5         387.5   
29       TXN  2013-12-01       -1083.0             -929.0        -154.0   
...      ...         ...           ...                ...           ...   
4470    ORLY  2015-12-01         144.6              201.7         -57.1   
4471    CDNS  2017-12-01         321.8               74.4         247.4   
4472      IT  2017-12-01        4915.9             3993.2         922.6   
4473    ANSS  2014-12-01          30.5              -50.7          81.3   
4474     CMI  2013-12-01        2180.0             1284.0         896.0   
4475     NUE  2014-12-01         412.6              247.8         164.8   
4476     GIS  2013-05-01        1561.2             1195.7         365.5   
4477    SCCO  2015-12-01        1199.3             1736.7        -537.4   
4478      EL  2014-06-01         723.6              156.1         567.5   
4479     LEN  2015-11-01        1496.3              797.6         698.8   
4480     XOM  2012-12-01        2743.0            -8173.0       10916.0   
4481     UNP  2011-12-01        2008.0             1193.0         815.0   
4482     CMS  2014-12-01        1769.0             1553.0         216.0   
4483     DUK  2016-12-01       11605.0            10335.0        1270.0   
4484     CPE  2013-12-01          45.8              -27.3          73.1   
4485    IDXX  2016-12-01          55.7               79.9         -24.2   
4486    STKL  2017-12-01        -147.4              -20.9        -126.5   
4487     AFL  2017-12-01        7398.0             3282.0        4116.0   
4488     BLK  2019-12-01        9049.0             7660.0        1389.0   
4489   DGICA  2019-12-01          91.1               38.9          52.1   
4490     BWA  2015-12-01        1597.7             1657.1         -59.4   
4491     DVA  2013-12-01      

In [3]:
classifications = data['Classification'] 
data = data.drop(['Company', 'Time', 'Classification'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Classification', classifications)
data


Total Assets
Done
Total Liabilities
Done
Total Equity
Done
Total Shares Out. on Filing Date
Done
Book Value / Share
Done
Tangible Book Value
Done
Tangible Book Value Per Share
Done
Total Debt
Done
Net Debt
Done
Total Revenues
Done
Operating Income
Done
Revenue Per Share
Done
Basic EPS
Done
Normalized Basic EPS
Done
Dividend Per Share
Done
EBITA
Done
EBIT
Done
Normalized Net Income
Done
Net Income
Done
Cash from Operations
Done
Cash from Investing
Done
Cash from Financing
Done
Net Change in Cash
Done
Levered Free Cash Flow
Done
Unlevered Free Cash Flow
Done
Free Cash Flow / Share
Done


Total Assets  Total Liabilities  Total Equity  \
0         0.569312           0.673112      0.530858   
1         0.557462           0.671981      0.508632   
2         0.559693           0.668188      0.519284   
3         0.558162           0.667093      0.517944   
4         0.559790           0.668336      0.519241   
5         0.563711           0.674297      0.517576   
6         0.548815           0.636383      0.548551   
7         0.560937           0.671061      0.517170   
8         0.553189           0.668362      0.505802   
9         0.560111           0.668193      0.520124   
10        0.559858           0.668253      0.519514   
11        0.551619           0.647500      0.536295   
12        0.562123           0.671848      0.518306   
13        0.559188           0.663439      0.525926   
14        0.561345           0.674058      0.513159   
15        0.874823           0.983345      0.650102   
16        0.557963           0.662474      0.524996   
17        0.511508           0.605177      0.523209   
18        0.561296           0.664860      0.527909   
19        0.580562           0.693826      0.520250   
20        0.559215           0.652288      0.543982   
21        0.558741           0.667783      0.518005   
22        0.564060           0.667829      0.528728   
23        0.562956           0.676908      0.511828   
24        0.558223           0.667465      0.517466   
25        0.559351           0.668380      0.518279   
26        0.559281           0.667486      0.519581   
27        0.560506           0.668170      0.520962   
28        0.560158           0.668054      0.520443   
29        0.555088           0.663805      0.517012   
...            ...                ...           ...   
4470      0.558921           0.668243      0.517626   
4471      0.559474           0.667744      0.519555   
4472      0.573817           0.683126      0.523834   
4473      0.558564           0.667253      0.518503   
4474      0.565275           0.672492      0.523666   
4475      0.559757           0.668424      0.519032   
4476      0.563343           0.672145      0.520304   
4477      0.562213           0.674269      0.514582   
4478      0.560728           0.668064      0.521584   
4479      0.563141           0.670583      0.522416   
4480      0.567033           0.635371      0.587162   
4481      0.564738           0.672135      0.523152   
4482      0.563992           0.673548      0.519356   
4483      0.594702           0.708019      0.526036   
4484      0.558612           0.667345      0.518451   
4485      0.558643           0.667765      0.517834   
4486      0.558009           0.667370      0.517186   
4487      0.581567           0.680334      0.544071   
4488      0.586722           0.697519      0.526790   
4489      0.558753           0.667604      0.518318   
4490      0.563457           0.673956      0.517611   
4491      0.561854           0.668547      0.523090   
4492      0.558625           0.667358      0.518457   
4493      0.549209           0.663143      0.506149   
4494      0.558545           0.667528      0.518017   
4495      0.626370           0.746483      0.528217   
4496      0.558001           0.667268      0.517333   
4497      0.559783           0.668987      0.518177   
4498      0.565072           0.681975      0.507943   
4499      0.553125           0.659165      0.520520   

      Total Shares Out. on Filing Date  Book Value / Share  \
0                             0.206247            0.001261   
1                             0.185579            0.001001   
2                             0.206252            0.001081   
3                             0.206256            0.001016   
4                             0.206239            0.001041   
5                             0.206171            0.001002   
6                             0.204582            0.001065   
7                             0.206085            0.001016   
8                             0.20595

In [4]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:26] # Parameters
y = NpMatrix[:,26] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

X: [[0.56931236 0.67311195 0.5308581  ... 0.55756272 0.56219563 0.43772384]
 [0.55746247 0.67198147 0.50863228 ... 0.54663096 0.5515018  0.43771544]
 [0.55969327 0.6681881  0.51928354 ... 0.54098986 0.54574963 0.43779811]
 ...
 [0.5597835  0.66898728 0.51817709 ... 0.54043122 0.54524518 0.43777422]
 [0.56507245 0.68197519 0.50794345 ... 0.54105975 0.5456177  0.43780909]
 [0.55312547 0.6591651  0.52051989 ... 0.55138498 0.55573384 0.4377148 ]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 26
Total Number of Training instances: 4500


[1. 1. 1. ... 1. 0. 0.]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 1.0
Total number of y values 4500


In [5]:
import tensorflow as tf
number_of_features = len(X[1])
model = tf.keras.Sequential([
    layers.Dense(units=676, activation='relu', input_shape=(number_of_features,)),
    layers.Dropout(0.2),
    layers.Dense(units = 1, activation='sigmoid')
])

model.compile(
            loss = 'binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
#give summary of model
model.summary()

scores = []
skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(x= X_train, y= y_train, batch_size= 5, epochs=1000) # Fit Model
    results = model.evaluate(x= X_test, y=y_test) # Evaluate Model Performance, Appending model accuracy to scores list
    scores.append(results[1]) # Appending model accuracy to scores list
    
scores = np.array(scores)
print(scores)
print("Mean Accuracy:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 676)               18252     
_________________________________________________________________
dropout (Dropout)            (None, 676)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 677       
Total params: 18,929
Trainable params: 18,929
Non-trainable params: 0
_________________________________________________________________
Train on 3599 samples
Epoch 1/1000
3599/3599 [==============================] - 4s 1ms/sample - loss: 0.6370 - accuracy: 0.6752
Epoch 2/1000
3599/3599 [==============================] - 2s 576us/sample - loss: 0.6340 - accuracy: 0.6752
Epoch 3/1000
3599/3599 [==============================] - 2s 544us/sample - loss: 0.6343 - accuracy: 0.6752
Epoch 4/1000
3599/3599 [==============

3599/3599 [==============================] - 3s 734us/sample - loss: 0.6281 - accuracy: 0.6777
Epoch 69/1000
3599/3599 [==============================] - 2s 623us/sample - loss: 0.6294 - accuracy: 0.6771
Epoch 70/1000
3599/3599 [==============================] - 2s 568us/sample - loss: 0.6294 - accuracy: 0.6774
Epoch 71/1000
3599/3599 [==============================] - 2s 493us/sample - loss: 0.6288 - accuracy: 0.6763
Epoch 72/1000
3599/3599 [==============================] - 2s 503us/sample - loss: 0.6280 - accuracy: 0.6771
Epoch 73/1000
3599/3599 [==============================] - 2s 442us/sample - loss: 0.6289 - accuracy: 0.6771
Epoch 74/1000
3599/3599 [==============================] - 2s 488us/sample - loss: 0.6280 - accuracy: 0.6771
Epoch 75/1000
3599/3599 [==============================] - 2s 532us/sample - loss: 0.6280 - accuracy: 0.6774
Epoch 76/1000
3599/3599 [==============================] - 2s 437us/sample - loss: 0.6272 - accuracy: 0.6774
Epoch 77/1000
3599/3599 [========

3599/3599 [==============================] - 2s 476us/sample - loss: 0.6260 - accuracy: 0.6785
Epoch 143/1000
3599/3599 [==============================] - 2s 443us/sample - loss: 0.6246 - accuracy: 0.6796
Epoch 144/1000
3599/3599 [==============================] - 2s 442us/sample - loss: 0.6244 - accuracy: 0.6807
Epoch 145/1000
3599/3599 [==============================] - 2s 437us/sample - loss: 0.6243 - accuracy: 0.6796
Epoch 146/1000
3599/3599 [==============================] - 2s 486us/sample - loss: 0.6240 - accuracy: 0.6796
Epoch 147/1000
3599/3599 [==============================] - 2s 448us/sample - loss: 0.6241 - accuracy: 0.6777
Epoch 148/1000
3599/3599 [==============================] - 2s 436us/sample - loss: 0.6248 - accuracy: 0.6796
Epoch 149/1000
3599/3599 [==============================] - 2s 472us/sample - loss: 0.6255 - accuracy: 0.6782
Epoch 150/1000
3599/3599 [==============================] - 2s 438us/sample - loss: 0.6251 - accuracy: 0.6796
Epoch 151/1000
3599/3599 

3599/3599 [==============================] - 2s 442us/sample - loss: 0.6221 - accuracy: 0.6794
Epoch 217/1000
3599/3599 [==============================] - 2s 501us/sample - loss: 0.6226 - accuracy: 0.6794
Epoch 218/1000
3599/3599 [==============================] - 2s 570us/sample - loss: 0.6233 - accuracy: 0.6782
Epoch 219/1000
3599/3599 [==============================] - 2s 486us/sample - loss: 0.6220 - accuracy: 0.6810
Epoch 220/1000
3599/3599 [==============================] - 2s 477us/sample - loss: 0.6228 - accuracy: 0.6791
Epoch 221/1000
3599/3599 [==============================] - 2s 440us/sample - loss: 0.6210 - accuracy: 0.6785
Epoch 222/1000
3599/3599 [==============================] - 2s 436us/sample - loss: 0.6201 - accuracy: 0.6802
Epoch 223/1000
3599/3599 [==============================] - 2s 476us/sample - loss: 0.6259 - accuracy: 0.6802
Epoch 224/1000
3599/3599 [==============================] - 2s 447us/sample - loss: 0.6221 - accuracy: 0.6807
Epoch 225/1000
3599/3599 

3599/3599 [==============================] - 2s 468us/sample - loss: 0.6215 - accuracy: 0.6807
Epoch 291/1000
3599/3599 [==============================] - 2s 447us/sample - loss: 0.6211 - accuracy: 0.6796
Epoch 292/1000
3599/3599 [==============================] - 2s 435us/sample - loss: 0.6214 - accuracy: 0.6794
Epoch 293/1000
3599/3599 [==============================] - 2s 440us/sample - loss: 0.6201 - accuracy: 0.6810
Epoch 294/1000
3599/3599 [==============================] - 2s 470us/sample - loss: 0.6187 - accuracy: 0.6821
Epoch 295/1000
3599/3599 [==============================] - 2s 445us/sample - loss: 0.6193 - accuracy: 0.6819
Epoch 296/1000
3599/3599 [==============================] - 2s 441us/sample - loss: 0.6194 - accuracy: 0.6830
Epoch 297/1000
3599/3599 [==============================] - 2s 473us/sample - loss: 0.6231 - accuracy: 0.6799
Epoch 298/1000
3599/3599 [==============================] - 2s 452us/sample - loss: 0.6199 - accuracy: 0.6810
Epoch 299/1000
3599/3599 

...

3601/3601 [==============================] - 4s 1ms/sample - loss: 0.6087 - accuracy: 0.6884
Epoch 743/1000
3601/3601 [==============================] - 2s 656us/sample - loss: 0.6038 - accuracy: 0.6884
Epoch 744/1000
3601/3601 [==============================] - 3s 773us/sample - loss: 0.6031 - accuracy: 0.6881
Epoch 745/1000
3601/3601 [==============================] - 3s 744us/sample - loss: 0.6014 - accuracy: 0.6887
Epoch 746/1000
3601/3601 [==============================] - 3s 744us/sample - loss: 0.6027 - accuracy: 0.6931
Epoch 747/1000
3601/3601 [==============================] - 3s 852us/sample - loss: 0.6051 - accuracy: 0.6895
Epoch 748/1000
3601/3601 [==============================] - 3s 818us/sample - loss: 0.6047 - accuracy: 0.6879
Epoch 749/1000
3601/3601 [==============================] - 3s 799us/sample - loss: 0.6024 - accuracy: 0.6884
Epoch 750/1000
3601/3601 [==============================] - 3s 721us/sample - loss: 0.6073 - accuracy: 0.6876
Epoch 751/1000
3601/3601 [=

3601/3601 [==============================] - 2s 501us/sample - loss: 0.6061 - accuracy: 0.6881
Epoch 817/1000
3601/3601 [==============================] - 2s 480us/sample - loss: 0.6064 - accuracy: 0.6887
Epoch 818/1000
3601/3601 [==============================] - 2s 449us/sample - loss: 0.6053 - accuracy: 0.6876
Epoch 819/1000
3601/3601 [==============================] - 2s 602us/sample - loss: 0.6061 - accuracy: 0.6865
Epoch 820/1000
3601/3601 [==============================] - 2s 649us/sample - loss: 0.6063 - accuracy: 0.6873
Epoch 821/1000
3601/3601 [==============================] - 2s 637us/sample - loss: 0.6025 - accuracy: 0.6898
Epoch 822/1000
3601/3601 [==============================] - 3s 705us/sample - loss: 0.6022 - accuracy: 0.6923
Epoch 823/1000
3601/3601 [==============================] - 3s 767us/sample - loss: 0.6016 - accuracy: 0.6881
Epoch 824/1000
3601/3601 [==============================] - 3s 803us/sample - loss: 0.6065 - accuracy: 0.6895
Epoch 825/1000
3601/3601 

3601/3601 [==============================] - 3s 699us/sample - loss: 0.6023 - accuracy: 0.6909
Epoch 891/1000
3601/3601 [==============================] - 2s 640us/sample - loss: 0.6087 - accuracy: 0.6876
Epoch 892/1000
3601/3601 [==============================] - 2s 677us/sample - loss: 0.6007 - accuracy: 0.6895
Epoch 893/1000
3601/3601 [==============================] - 2s 691us/sample - loss: 0.6007 - accuracy: 0.6895
Epoch 894/1000
3601/3601 [==============================] - 2s 674us/sample - loss: 0.6004 - accuracy: 0.6876
Epoch 895/1000
3601/3601 [==============================] - 3s 731us/sample - loss: 0.5998 - accuracy: 0.6881
Epoch 896/1000
3601/3601 [==============================] - 3s 713us/sample - loss: 0.6025 - accuracy: 0.6890
Epoch 897/1000
3601/3601 [==============================] - 2s 685us/sample - loss: 0.6063 - accuracy: 0.6868
Epoch 898/1000
3601/3601 [==============================] - 2s 655us/sample - loss: 0.6039 - accuracy: 0.6884
Epoch 899/1000
3601/3601 

3601/3601 [==============================] - 2s 654us/sample - loss: 0.6058 - accuracy: 0.6904
Epoch 964/1000
3601/3601 [==============================] - 2s 693us/sample - loss: 0.6064 - accuracy: 0.6856
Epoch 965/1000
3601/3601 [==============================] - 2s 643us/sample - loss: 0.6043 - accuracy: 0.6909
Epoch 966/1000
3601/3601 [==============================] - 3s 701us/sample - loss: 0.6036 - accuracy: 0.6856
Epoch 967/1000
3601/3601 [==============================] - 2s 663us/sample - loss: 0.6044 - accuracy: 0.6884
Epoch 968/1000
3601/3601 [==============================] - 2s 690us/sample - loss: 0.6024 - accuracy: 0.6887
Epoch 969/1000
3601/3601 [==============================] - 2s 677us/sample - loss: 0.6001 - accuracy: 0.6887
Epoch 970/1000
3601/3601 [==============================] - 2s 649us/sample - loss: 0.6037 - accuracy: 0.6904
Epoch 971/1000
3601/3601 [==============================] - 2s 693us/sample - loss: 0.6008 - accuracy: 0.6912
Epoch 972/1000
3601/3601 

[0.6870144  0.6792453  0.6888889  0.68631816 0.6985539 ]
Mean Accuracy: 0.68800414
Standard Deviation of the mean: 0.006206773


In [6]:
scores = np.array(scores)
print(scores)
print("Mean Accuracy:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[0.6870144  0.6792453  0.6888889  0.68631816 0.6985539 ]
Mean Accuracy: 0.68800414
Standard Deviation of the mean: 0.006206773
